### Importing Libraries:

In [1]:
import numpy as np
import pandas as pd
import difflib  # this used to find the closest mach if the user enterd the movie name wrong.
from sklearn.feature_extraction.text import TfidfVectorizer # this used to convert text to numbers
from sklearn.metrics.pairwise import cosine_similarity # this is the recommendation algorithm

### Loading the data:

In [2]:
df = pd.read_csv("movies.csv")
df.head() #print first 5 rows

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


### Data Pre_processing & Cleaning

In [3]:
# check how many rows and columns we have:
df.shape

(4803, 24)

In [4]:
# getting a statistical summary:
df.describe().round()

,index,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4803.0,4803.0,4803.0,4803.0,4.803000e+03,4801.0,4803.0,4803.0
mean,2401.0,29045040.0,57165.0,21.0,8.226064e+07,107.0,6.0,690.0
std,1387.0,40722391.0,88695.0,32.0,1.628571e+08,23.0,1.0,1235.0
min,0.0,0.0,5.0,0.0,0.000000e+00,0.0,0.0,0.0
25%,1200.0,790000.0,9014.0,5.0,0.000000e+00,94.0,6.0,54.0
50%,2401.0,15000000.0,14629.0,13.0,1.917000e+07,103.0,6.0,235.0
75%,3602.0,40000000.0,58610.0,28.0,9.291719e+07,118.0,7.0,737.0
max,4802.0,380000000.0,459488.0,876.0,2.787965e+09,338.0,10.0,13752.0


In [5]:
# getting more information about the data:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [6]:
# check for missing values:
#df.isnull().sum()

In [7]:
# drop missing values:
#df.dropna(inplace=True)

#check for missing values again:
#df.isnull().sum()

In [8]:
# check for duplicated values:
df.duplicated().sum()

0

### Features selection & processing

In [9]:
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [10]:
# now we select the most relevant columns or features for our recommendation system:

selected_features = ["genres","keywords","tagline","cast","director","overview","production_companies"]
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director', 'overview', 'production_companies']


In [11]:
# fulling missing null values with null string in the choosen columns: 

for feature in selected_features:
    df[feature] = df[feature].fillna("")

In [12]:
# combine all selected features togther:

combined_features = df["genres"]+" "+df["keywords"]+" "+df["tagline"]+" "+df["cast"]+" "+df["director"]+" "+df["overview"]+" "+df["production_companies"]

print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [13]:
# now converting the text data to features vectors aka(numerical data):
vectorizer  = TfidfVectorizer()

vectoried_features = vectorizer.fit_transform(combined_features)

print(vectoried_features)

  (0, 3920)	0.17327889069926256
  (0, 22268)	0.17327889069926256
  (0, 3313)	0.12136395207127132
  (0, 14702)	0.1048024129681479
  (0, 13974)	0.11197305145417999
  (0, 2587)	0.09176446226126546
  (0, 11621)	0.07749073918929551
  (0, 16298)	0.0783106976007871
  (0, 35221)	0.09077542775554663
  (0, 2466)	0.13699190582135137
  (0, 19270)	0.09702793745452735
  (0, 26790)	0.11197305145417999
  (0, 15793)	0.10862986849351187
  (0, 19689)	0.12940567781728674
  (0, 25140)	0.09692673052493186
  (0, 10651)	0.14850917004238112
  (0, 6033)	0.11053343245396427
  (0, 6280)	0.047391309353417795
  (0, 28253)	0.14979788486799298
  (0, 6321)	0.02866019019358485
  (0, 26209)	0.1450336315673668
  (0, 16144)	0.12459035683599573
  (0, 8179)	0.08964315446436125
  (0, 34673)	0.13050571897807273
  (0, 7875)	0.08821682148183961
  :	:
  (4802, 26024)	0.0553626524377205
  (4802, 37761)	0.0571114479660939
  (4802, 31401)	0.047163087925317385
  (4802, 18483)	0.07763892028464114
  (4802, 18267)	0.10175660085342444
 

### Cosine Similarity

In [14]:
# now we will find the similarty score for each movie using cosine_similarty:

# the cosine_similarty will try to compare each movie to all the other movies, and based on that return 
#>>> us a similarty score about how each movie is similar to the others based on that comparsion.

similarty = cosine_similarity(vectoried_features)

print(similarty) # this will print the similarty score

[[1.         0.05596506 0.04124366 ... 0.03534218 0.0276738  0.0164769 ]
 [0.05596506 1.         0.04880217 ... 0.05138505 0.02877342 0.02877794]
 [0.04124366 0.04880217 1.         ... 0.03009804 0.04432021 0.01685446]
 ...
 [0.03534218 0.05138505 0.03009804 ... 1.         0.04074648 0.03655312]
 [0.0276738  0.02877342 0.04432021 ... 0.04074648 1.         0.02946268]
 [0.0164769  0.02877794 0.01685446 ... 0.03655312 0.02946268 1.        ]]


### Movies recommendation system

#### Here we will ask the user to give us a movie name, and we will suggest similar movies to him.

In [15]:
def recommend_movie():
    user_movie = input("Please enter your favourite movie name: ")
    
    # Creat a list contains the movies names from the dataset
    movies_names = df["title"].tolist()

    # now finding the closest match for the user input
    find_match = difflib.get_close_matches(user_movie,movies_names)
    
    #"find_match" gave us a list of movies similar to the user input, we will slice the first movie to find
    #>>> it's index
    movie_slice = find_match[0]

    # now we find the index of the sliced movie

    movie_index = df[df.title == movie_slice]["index"].values[0]
    
    #now we will find the similarty score for all movies compared to the sliced movie

    sim_score  = list(enumerate(similarty[movie_index]))
    
    # now we sort the movies based on similarty score

    sorted_sim = sorted(sim_score, key = lambda x:x[1], reverse=True)
    
    
    # now we print a list of movies similar to the ser given movie

    print("Movies suggested for you : \n")

    i = 1 

    for movie in sorted_sim:
    
        index = movie[0]
    
        # code below will find the movie title from it's index number
        title_from_index = df[df.index==index]["title"].values[0]
    
        if (i<30):
            print(i, ".",title_from_index)
            i= i+1

In [16]:
# uncomment to use the recommendation system

#recommend_movie() #the recommendation system